In [61]:
import pandas as pd
import numpy as np
import math

In [3]:
df_vehicles = pd.read_csv("data/Traffic_Crashes_Vehicles.csv")
df_people = pd.read_csv("data/Traffic_Crashes_People.csv")

C:\Users\Owner\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (20,39,40,41,43,47,48,49,52,54,57,58,60,70) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
C:\Users\Owner\anaconda3\envs\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3145: DtypeWarning: Columns (19,23,24,25,28) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [100]:
df_people_sample = df_people.sample(n=300000, random_state=42)

In [101]:
df_people_sample

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID,CRASH_DATE,SEAT_NO,CITY,STATE,ZIPCODE,SEX,...,EMS_RUN_NO,DRIVER_ACTION,DRIVER_VISION,PHYSICAL_CONDITION,PEDPEDAL_ACTION,PEDPEDAL_VISIBILITY,PEDPEDAL_LOCATION,BAC_RESULT,BAC_RESULT VALUE,CELL_PHONE_USE
1459795,O1570236,DRIVER,9f3300f82fb71bd87e1a388f1190525c5cd80462c39dc6...,1493761.0,05/06/2023 08:50:00 PM,NaN,NaN,NaN,NaN,F,...,NaN,FOLLOWED TOO CLOSELY,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1569870,O1596587,DRIVER,cc52c13e5510187a8bc57c42ef1b62b32333e908163e07...,1519222.0,06/12/2023 10:30:00 PM,NaN,CHICAGO,IL,60659,M,...,NaN,FAILED TO YIELD,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
550756,O130199,DRIVER,012f0e85941af3ed53e800bd3fb2dee18f91b068812fb5...,130114.0,02/26/2017 12:00:00 PM,NaN,NaN,NaN,NaN,X,...,NaN,IMPROPER PASSING,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1035836,O67422,DRIVER,c4619ca2a35bd07020e8601e53b5645aaed6224db299b6...,65208.0,08/16/2016 07:20:00 PM,NaN,CHICAGO,IL,60636,M,...,NaN,NONE,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
1891337,O1832359,DRIVER,54f605aad8e9a7b31203c17ca89a2b75ab370001a83c71...,1745443.0,06/01/2024 03:34:00 PM,NaN,CHICAGO,IL,60660,M,...,NaN,NONE,UNKNOWN,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1337827,P88986,PASSENGER,cefee604464486586bf19f2bb5562070957571ad5925be...,385204.0,06/02/2018 07:00:00 PM,3.0,NaN,NaN,NaN,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1529038,O1883597,DRIVER,a4b716120b9b9a6faeaf02fa7585145d47a2a0690e23c1...,1794399.0,08/10/2024 06:15:00 PM,NaN,NaN,NaN,NaN,M,...,NaN,OTHER,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
510360,O1342669,DRIVER,c45fe908b1b5b115d909af70b71b8fecaf929c1f83ea92...,1275601.0,05/21/2022 12:50:00 PM,NaN,NaN,NaN,NaN,X,...,NaN,UNKNOWN,UNKNOWN,UNKNOWN,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN
907137,O527730,DRIVER,06d845100c4a605e5407d31b63d3840ac23f1e3b89f7ec...,503988.0,11/23/2018 09:45:00 AM,NaN,CHICAGO,IL,60644,F,...,NaN,FAILED TO YIELD,NOT OBSCURED,NORMAL,NaN,NaN,NaN,TEST NOT OFFERED,NaN,NaN


In [103]:
df_final = df_people_sample.merge(df_vehicles, how="left", left_on="CRASH_RECORD_ID", right_on="CRASH_RECORD_ID")

In [104]:
df_final

,PERSON_ID,PERSON_TYPE,CRASH_RECORD_ID,VEHICLE_ID_x,CRASH_DATE_x,SEAT_NO,CITY,STATE,ZIPCODE,SEX,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,O1570236,DRIVER,9f3300f82fb71bd87e1a388f1190525c5cd80462c39dc6...,1493761.0,05/06/2023 08:50:00 PM,NaN,NaN,NaN,NaN,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,O1570236,DRIVER,9f3300f82fb71bd87e1a388f1190525c5cd80462c39dc6...,1493761.0,05/06/2023 08:50:00 PM,NaN,NaN,NaN,NaN,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,O1596587,DRIVER,cc52c13e5510187a8bc57c42ef1b62b32333e908163e07...,1519222.0,06/12/2023 10:30:00 PM,NaN,CHICAGO,IL,60659,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,O1596587,DRIVER,cc52c13e5510187a8bc57c42ef1b62b32333e908163e07...,1519222.0,06/12/2023 10:30:00 PM,NaN,CHICAGO,IL,60659,M,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,O130199,DRIVER,012f0e85941af3ed53e800bd3fb2dee18f91b068812fb5...,130114.0,02/26/2017 12:00:00 PM,NaN,NaN,NaN,NaN,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
629569,O1342669,DRIVER,c45fe908b1b5b115d909af70b71b8fecaf929c1f83ea92...,1275601.0,05/21/2022 12:50:00 PM,NaN,NaN,NaN,NaN,X,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
629570,O527730,DRIVER,06d845100c4a605e5407d31b63d3840ac23f1e3b89f7ec...,503988.0,11/23/2018 09:45:00 AM,NaN,CHICAGO,IL,60644,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
629571,O527730,DRIVER,06d845100c4a605e5407d31b63d3840ac23f1e3b89f7ec...,503988.0,11/23/2018 09:45:00 AM,NaN,CHICAGO,IL,60644,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
629572,P37948,PASSENGER,dd3ea43ebaf3183b021c3a4669996727a6ab0acb9c55af...,175734.0,06/30/2017 03:00:00 PM,3.0,CHICAGO,IL,60647,F,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [99]:
count = 0
for column in df_final.iloc[0]:
    print(df_final.columns[count], ":", column, ",", df_final.iloc[1].iloc[count])
    count += 1

PERSON_ID : O1570236 , O1570236
PERSON_TYPE : DRIVER , DRIVER
CRASH_RECORD_ID : 9f3300f82fb71bd87e1a388f1190525c5cd80462c39dc694354c269cf82ee410274004bd2f1badc25669add099ea8c4ec7cf4854bace4f6ffe897c73497fcb5e , 9f3300f82fb71bd87e1a388f1190525c5cd80462c39dc694354c269cf82ee410274004bd2f1badc25669add099ea8c4ec7cf4854bace4f6ffe897c73497fcb5e
VEHICLE_ID_x : 1493761.0 , 1493761.0
CRASH_DATE_x : 05/06/2023 08:50:00 PM , 05/06/2023 08:50:00 PM
SEAT_NO : nan , nan
CITY : nan , nan
STATE : nan , nan
ZIPCODE : nan , nan
SEX : F , F
AGE : nan , nan
DRIVERS_LICENSE_STATE : nan , nan
DRIVERS_LICENSE_CLASS : nan , nan
SAFETY_EQUIPMENT : USAGE UNKNOWN , USAGE UNKNOWN
AIRBAG_DEPLOYED : DEPLOYMENT UNKNOWN , DEPLOYMENT UNKNOWN
EJECTION : NONE , NONE
INJURY_CLASSIFICATION : NO INDICATION OF INJURY , NO INDICATION OF INJURY
HOSPITAL : nan , nan
EMS_AGENCY : nan , nan
EMS_RUN_NO : nan , nan
DRIVER_ACTION : FOLLOWED TOO CLOSELY , FOLLOWED TOO CLOSELY
DRIVER_VISION : UNKNOWN , UNKNOWN
PHYSICAL_CONDITION : UNK

In [105]:
df_people_sample.to_csv("data/People_Sample.csv")
#df_final.to_csv("data/Final_Data.csv")